<a href="https://colab.research.google.com/github/volpo/PG_YELP_GOOGLE_MAPS/blob/main/pruebas_de_carga_google_stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

modelos de regresión


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

In [ ]:

    '''# Create a new DataFrame with the unique categories
    df_category = pd.DataFrame({'category': stores['category'].apply(list).explode().unique()})

    #aux_df = df_stores['category'].apply(list).explode().unique()


    # Split the categories into separate rows
    splitted_df = aux_df['category'].explode()

    # Get the unique categories
    unique_categories = splitted_df.unique()

    # Create a new DataFrame with the unique categories
    df_category = pd.DataFrame({'category': unique_categories})


    #aux_df.explode().unique()
    #df_category.head()

    return df_category'''

In [1]:
#Librerias Requeridas
!pip install cassandra-driver

from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, dict_factory

import pandas as pd

cluster = Cluster(['186.87.6.161'], port='9042', protocol_version = 5) #IP del servidor y el puerto estandar de cassandra 9042
session = cluster.connect('henry')
session.row_factory = dict_factory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 63.3 MB/s eta 0:00:00


In [3]:

indice_inicio = 0
tamaño_particion = 5000

    # Calcular el índice final
indice_fin = indice_inicio + tamaño_particion

Estados = ['New York','Texas', 'Nevada', 'California', 'Florida']

dictionary = {
            'gmap_id':[],
            'state': [],
            'user_id':[],
            'name': [],
            'time':[],
            'rating': [],
            'text':[],
            'resp': []}

for estado in Estados:
# Construct the query
    query = f"""SELECT gmap_id, state, user_id, name, time, rating, text, resp
                FROM reviews WHERE state = '{estado}'
                """
    statement = SimpleStatement(query,fetch_size = 5000)
    resp = session.execute(statement,timeout=None)


    for i, row in enumerate(resp):
        # Verificar si el índice actual está dentro del rango de la partición
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

reviews_texas = pd.DataFrame(dictionary)
reviews_texas.head()

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
from cassandra.query import SimpleStatement

# Definir el tamaño de la partición y los estados
tamaño_particion = 5000
estados = ['New York', 'Texas', 'Nevada', 'California', 'Florida']

# Crear un DataFrame vacío para almacenar los resultados
reviews_combined = pd.DataFrame()

# Calcular el índice de inicio
indice_inicio = 0

for estado in estados:
    # Calcular el índice final
    indice_fin = indice_inicio + tamaño_particion

    # Construir la consulta CQL
    query = f"""
    SELECT gmap_id, state, user_id, name, time, rating, text, resp
    FROM reviews WHERE state = '{estado}' ALLOW FILTERING
    """

    statement = SimpleStatement(query, fetch_size=5000)

    # Ejecutar la consulta en la base de datos Cassandra
    resp = session.execute(statement, timeout=None)

    # Crear un diccionario para almacenar los datos del estado actual
    dictionary = {
        'gmap_id': [],
        'state': [],
        'user_id': [],
        'name': [],
        'time': [],
        'rating': [],
        'text': [],
        'resp': []
    }

    # Iterar a través de los resultados y recopilar los datos en el diccionario
    for i, row in enumerate(resp):
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

    # Crear un DataFrame para el estado actual
    reviews_state = pd.DataFrame(dictionary)

    # Concatenar el DataFrame del estado actual con el DataFrame combinado
    reviews_combined = pd.concat([reviews_combined, reviews_state], ignore_index=True)

    # Actualizar el índice de inicio para el próximo estado
    indice_inicio = indice_fin

# Ahora, reviews_combined contiene todos los datos de los estados especificados
# Puedes acceder a los datos combinados utilizando reviews_combined.head() u otras operaciones de Pandas


In [4]:
indice_inicio = 0
tamaño_particion = 5000

# Inicializar una variable para el recuento total de filas procesadas
total_filas_procesadas = 0

# Calcular el índice final
indice_fin = indice_inicio + tamaño_particion

Estados = ['New York', 'Texas', 'Nevada', 'California', 'Florida']

dictionary = {
    'gmap_id': [],
    'state': [],
    'user_id': [],
    'name': [],
    'time': [],
    'rating': [],
    'text': [],
    'resp': []
}

for estado in Estados:
    # Verificar si ya se procesaron 5000 filas en total
    if total_filas_procesadas >= 5000:
        break

    # Construct the query
    query = f"""SELECT gmap_id, state, user_id, name, time, rating, text, resp
                FROM reviews WHERE state = '{estado}'
                """
    statement = SimpleStatement(query, fetch_size=5000)
    resp = session.execute(statement, timeout=None)

    for i, row in enumerate(resp):
        # Verificar si el índice actual está dentro del rango de la partición
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

            # Incrementar el recuento total de filas procesadas
            total_filas_procesadas += 1

    # Actualizar el índice de inicio para la próxima partición
    indice_inicio = indice_fin

# Crear el DataFrame
reviews_texas = pd.DataFrame(dictionary)
reviews_texas.head()


KeyboardInterrupt: ignored

In [5]:
indice_inicio = 0
tamaño_particion = 5000

# Calcular el índice final
indice_fin = indice_inicio + tamaño_particion

Estados = ['New York', 'Texas', 'Nevada', 'California', 'Florida']

dictionary = {
    'gmap_id': [],
    'state': [],
    'user_id': [],
    'name': [],
    'time': [],
    'rating': [],
    'text': [],
    'resp': []
}

# Inicializar una variable para el recuento total de filas procesadas
total_filas_procesadas = 0

for estado in Estados:
    # Verificar si ya se procesaron 5000 filas en total
    if total_filas_procesadas >= 5000:
        break

    # Construct the query
    query = f"""SELECT gmap_id, state, user_id, name, time, rating, text, resp
                FROM reviews WHERE state = '{estado}'
                """
    statement = SimpleStatement(query, fetch_size=5000)
    resp = session.execute(statement, timeout=None)

    for i, row in enumerate(resp):
        # Verificar si el índice actual está dentro del rango de la partición
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

    # Incrementar el recuento total de filas procesadas después de cada consulta
    total_filas_procesadas += min(indice_fin, len(resp))

    # Actualizar el índice de inicio para la próxima partición
    indice_inicio = indice_fin

# Crear el DataFrame
reviews_texas = pd.DataFrame(dictionary)
reviews_texas.head()


KeyboardInterrupt: ignored

In [ ]:


'''if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test'''


'''@data_loader'''

categories_searched = ['Restaurant',
                        'restaurant',
                        'Bar',
                        'bar',
                        'Deli',
                        'Grocery',
                        'Coffee',
                        'Bakery',
                        'Sandwich']

dictionary = {'gmap_id':[],
                'name': [],
                'address': [],
                'latitude': [],
                'longitude': [],
                'category':[],
                'misc':[]}



for category_searched in categories_searched:
# Construct the query
    query = f"""SELECT gmap_id, name, address, latitude, longitude, category, misc
                FROM stores
                WHERE category CONTAINS '{category_searched}'
                """
    statement = SimpleStatement(query, fetch_size=300)
    answer = session.execute(statement, timeout=None)

    for row in answer:
        dictionary['gmap_id'].append(row['gmap_id'])
        dictionary['name'].append(row['name'])
        dictionary['address'].append(row['address'])
        dictionary['latitude'].append(row['latitude'])
        dictionary['longitude'].append(row['longitude'])
        # Convert the SortedSet to a comma-separated string
        category_str = ', '.join(row['category'])
        dictionary['category'].append(category_str)
        dictionary['misc'].append(row['misc'])


df_stores = pd.DataFrame(dictionary)

print(df_stores)


'''
@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'
'''


                                      gmap_id  \
0       0x87f38b6db3bf5557:0xb793ff26fbedb7fe   
1       0x80c8bf86a8b9a3f1:0xacfe57cd1919e249   
2       0x808506614c768671:0xde51b62058db3bce   
3       0x8842ecec5931b86b:0x703640198411596d   
4       0x87e030ec559cac7f:0x8f04ab05959695e6   
...                                       ...   
157439  0x876a706c24f78761:0xda56387cfc9157b7   
157440  0x54906ab2f6d99cff:0x2951ee439830694f   
157441  0x88e7790ad915b3bf:0xe29ffb688b004513   
157442  0x89c25a263dab771f:0xe92ae29003b3656d   
157443  0x89ba83163154720d:0x8656d643ae68b6a4   

                                       name  \
0                                     Shell   
1                   Ember's Grill + Spirits   
2                                  La Toque   
3                         Godfather's Pizza   
4                                    Subway   
...                                     ...   
157439                    Mountain Cupcakes   
157440                         Conf

'\n@test\ndef test_output(output, *args) -> None:\n    """\n    Template code for testing the output of the block.\n    """\n    assert output is not None, \'The output is undefined\'\n'

In [ ]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157444 entries, 0 to 157443
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   gmap_id    157444 non-null  object 
 1   name       157444 non-null  object 
 2   address    156787 non-null  object 
 3   latitude   157444 non-null  float64
 4   longitude  157444 non-null  float64
 5   misc       157444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 7.2+ MB


In [ ]:
df_stores.head()

,gmap_id,name,address,latitude,longitude,category,misc
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,"ATM, Car wash, Convenience store, Gas station,...",{'Accessibility': ['Wheelchair accessible entr...
1,0x80c8bf86a8b9a3f1:0xacfe57cd1919e249,Ember's Grill + Spirits,"Ember's Grill + Spirits, 740 S Rampart Blvd, L...",36.163380,-115.289413,"Bar, Restaurant","{'Service options': ['Delivery'], 'Highlights'..."
2,0x808506614c768671:0xde51b62058db3bce,La Toque,"La Toque, 1314 McKinstry St, Napa, CA 94559",38.304173,-122.283989,"French restaurant, Restaurant","{'Service options': ['Outdoor seating', 'Dine-..."
3,0x8842ecec5931b86b:0x703640198411596d,Godfather's Pizza,"Godfather's Pizza, 233 Lexington Rd, Lancaster...",37.625710,-84.580132,"Buffet restaurant, Delivery Restaurant, Italia...","{'Service options': ['Delivery'], 'Popular for..."
4,0x87e030ec559cac7f:0x8f04ab05959695e6,Subway,"Subway, 330 N 1st St, Cuba, IL 61427",40.496239,-90.198875,"Caterer, Fast food restaurant, Restaurant, San...","{'Service options': ['Curbside pickup', 'Takeo..."


In [ ]:
# Crear un DataFrame con categorías únicas separadas por palabras
unique_categories = df_stores['category'].apply(lambda x: x.split(', ')).explode().unique()
unique_categories



array(['ATM', 'Car wash', 'Convenience store', ..., 'Handicraft fair',
       'Handicrafts wholesaler', 'Dry cleaner'], dtype=object)

In [ ]:
df_category = pd.DataFrame({'category': unique_categories})
df_category

NameError: ignored

In [ ]:
#eexportar a un csv las categorias

df_category.to_csv('categories', index= False)


**Get Categories id**



The second step is to get the categories id of each store


In [ ]:
# Crear un diccionario para mapear categorías a índices
category_index_map = {category: index for index, category in enumerate(df_category['category'])}
category_index_map

{'ATM': 0,
 'Car wash': 1,
 'Convenience store': 2,
 'Gas station': 3,
 'Restaurant': 4,
 'Bar': 5,
 'French restaurant': 6,
 'Buffet restaurant': 7,
 'Delivery Restaurant': 8,
 'Italian restaurant': 9,
 'Pizza restaurant': 10,
 'Caterer': 11,
 'Fast food restaurant': 12,
 'Sandwich shop': 13,
 'Takeout Restaurant': 14,
 'Brunch restaurant': 15,
 'Cocktail bar': 16,
 'Family restaurant': 17,
 'Fine dining restaurant': 18,
 'Lunch restaurant': 19,
 'Mediterranean restaurant': 20,
 'Pizza delivery': 21,
 'Japanese restaurant': 22,
 'Sushi restaurant': 23,
 'American restaurant': 24,
 'Hamburger restaurant': 25,
 'Hot dog restaurant': 26,
 'Hot dog stand': 27,
 'Chinese restaurant': 28,
 'Thai restaurant': 29,
 'Latin American restaurant': 30,
 'Mexican restaurant': 31,
 'Alternative fuel station': 32,
 'Electric vehicle charging station': 33,
 'Bar & grill': 34,
 'Amusement center': 35,
 'Event venue': 36,
 'Video arcade': 37,
 'Delivery service': 38,
 'Asian restaurant': 39,
 'Southeast

In [ ]:
df_stores['category'] = df_stores['category'].apply(lambda x: x.split(', '))

df_stores.head()

,gmap_id,name,address,latitude,longitude,category,misc
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,"[ATM, Car wash, Convenience store, Gas station...",{'Accessibility': ['Wheelchair accessible entr...
1,0x80c8bf86a8b9a3f1:0xacfe57cd1919e249,Ember's Grill + Spirits,"Ember's Grill + Spirits, 740 S Rampart Blvd, L...",36.163380,-115.289413,"[Bar, Restaurant]","{'Service options': ['Delivery'], 'Highlights'..."
2,0x808506614c768671:0xde51b62058db3bce,La Toque,"La Toque, 1314 McKinstry St, Napa, CA 94559",38.304173,-122.283989,"[French restaurant, Restaurant]","{'Service options': ['Outdoor seating', 'Dine-..."
3,0x8842ecec5931b86b:0x703640198411596d,Godfather's Pizza,"Godfather's Pizza, 233 Lexington Rd, Lancaster...",37.625710,-84.580132,"[Buffet restaurant, Delivery Restaurant, Itali...","{'Service options': ['Delivery'], 'Popular for..."
4,0x87e030ec559cac7f:0x8f04ab05959695e6,Subway,"Subway, 330 N 1st St, Cuba, IL 61427",40.496239,-90.198875,"[Caterer, Fast food restaurant, Restaurant, Sa...","{'Service options': ['Curbside pickup', 'Takeo..."


In [ ]:
# Create a new column 'category_id' in the original DataFrame
df_stores['category_id'] = df_stores['category'].apply(lambda x: [category_index_map[category] for category in x])

df_stores.head()

,gmap_id,name,address,latitude,longitude,category,misc,category_id
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,"[ATM, Car wash, Convenience store, Gas station...",{'Accessibility': ['Wheelchair accessible entr...,"[0, 1, 2, 3, 4]"
1,0x80c8bf86a8b9a3f1:0xacfe57cd1919e249,Ember's Grill + Spirits,"Ember's Grill + Spirits, 740 S Rampart Blvd, L...",36.163380,-115.289413,"[Bar, Restaurant]","{'Service options': ['Delivery'], 'Highlights'...","[5, 4]"
2,0x808506614c768671:0xde51b62058db3bce,La Toque,"La Toque, 1314 McKinstry St, Napa, CA 94559",38.304173,-122.283989,"[French restaurant, Restaurant]","{'Service options': ['Outdoor seating', 'Dine-...","[6, 4]"
3,0x8842ecec5931b86b:0x703640198411596d,Godfather's Pizza,"Godfather's Pizza, 233 Lexington Rd, Lancaster...",37.625710,-84.580132,"[Buffet restaurant, Delivery Restaurant, Itali...","{'Service options': ['Delivery'], 'Popular for...","[7, 8, 9, 10, 4]"
4,0x87e030ec559cac7f:0x8f04ab05959695e6,Subway,"Subway, 330 N 1st St, Cuba, IL 61427",40.496239,-90.198875,"[Caterer, Fast food restaurant, Restaurant, Sa...","{'Service options': ['Curbside pickup', 'Takeo...","[11, 12, 4, 13, 14]"


In [ ]:
# Explode the 'category_id' column to create separate rows for each category_id
exploded_df = df_stores.explode('category_id')

In [ ]:
exploded_df

,gmap_id,name,address,latitude,longitude,category,misc,category_id
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,"[ATM, Car wash, Convenience store, Gas station...",{'Accessibility': ['Wheelchair accessible entr...,0
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,"[ATM, Car wash, Convenience store, Gas station...",{'Accessibility': ['Wheelchair accessible entr...,1
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,"[ATM, Car wash, Convenience store, Gas station...",{'Accessibility': ['Wheelchair accessible entr...,2
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,"[ATM, Car wash, Convenience store, Gas station...",{'Accessibility': ['Wheelchair accessible entr...,3
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,"[ATM, Car wash, Convenience store, Gas station...",{'Accessibility': ['Wheelchair accessible entr...,4
...,...,...,...,...,...,...,...,...
157443,0x89ba83163154720d:0x8656d643ae68b6a4,Ethereal Cupcakes And Coffee Shoppe,"Ethereal Cupcakes And Coffee Shoppe, 3150 Alla...",37.044067,-76.394806,"[Bakery, Coffee shop, Live music venue, Vegan ...","{'Service options': ['Outdoor seating', 'Deliv...",73
157443,0x89ba83163154720d:0x8656d643ae68b6a4,Ethereal Cupcakes And Coffee Shoppe,"Ethereal Cupcakes And Coffee Shoppe, 3150 Alla...",37.044067,-76.394806,"[Bakery, Coffee shop, Live music venue, Vegan ...","{'Service options': ['Outdoor seating', 'Deliv...",52
157443,0x89ba83163154720d:0x8656d643ae68b6a4,Ethereal Cupcakes And Coffee Shoppe,"Ethereal Cupcakes And Coffee Shoppe, 3150 Alla...",37.044067,-76.394806,"[Bakery, Coffee shop, Live music venue, Vegan ...","{'Service options': ['Outdoor seating', 'Deliv...",74
157443,0x89ba83163154720d:0x8656d643ae68b6a4,Ethereal Cupcakes And Coffee Shoppe,"Ethereal Cupcakes And Coffee Shoppe, 3150 Alla...",37.044067,-76.394806,"[Bakery, Coffee shop, Live music venue, Vegan ...","{'Service options': ['Outdoor seating', 'Deliv...",42



**Create Pivot table**

The third step is to create a pivot table representing the N:M relation between Store and Category

In [ ]:

# Create a new DataFrame with the desired columns
stores_categories = exploded_df[['gmap_id', 'category_id']]

stores_categories.head()


,gmap_id,category_id
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,0
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,1
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,2
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,3
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,4


In [ ]:
#exportarmos a csv el df de categorias
stores_categories.to_csv('pivot_categories', index= True)

**Drop non-normalized columns**

The last step is to drop the columns we have already normalized by the previous steps

In [ ]:
# Drop the 'category' and 'category_id' columns from the original DataFrame
df_stores = df_stores.drop(['category', 'category_id'], axis=1)

df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157444 entries, 0 to 157443
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   gmap_id    157444 non-null  object 
 1   name       157444 non-null  object 
 2   address    156787 non-null  object 
 3   latitude   157444 non-null  float64
 4   longitude  157444 non-null  float64
 5   misc       157444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 7.2+ MB


In [ ]:

df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157444 entries, 0 to 157443
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   gmap_id    157444 non-null  object 
 1   name       157444 non-null  object 
 2   address    156787 non-null  object 
 3   latitude   157444 non-null  float64
 4   longitude  157444 non-null  float64
 5   misc       157444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 7.2+ MB


In [ ]:
statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'Texas';", fetch_size = 1000)
resp = session.execute(statement, timeout=None)


dictionary = {'gmap_id':[], 'state': []}


for row in resp:
  dictionary['gmap_id'].append(row['gmap_id'])
  dictionary['state'].append(row['state'])


gmap_id = pd.DataFrame(dictionary)

gmap_id.head()


,gmap_id,state
0,0x865689322c0fb279:0xa457523bbcff9910,Texas
1,0x865689322c0fb279:0xa457523bbcff9910,Texas
2,0x865689322c0fb279:0xa457523bbcff9910,Texas
3,0x865689322c0fb279:0xa457523bbcff9910,Texas
4,0x865689322c0fb279:0xa457523bbcff9910,Texas


In [ ]:
gmap_id_TX = gmap_id

In [ ]:
gmap_id_TX.nunique()

gmap_id    67740
state          1
dtype: int64

In [ ]:
gmap_id_TX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2235415 entries, 0 to 2235414
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   gmap_id  object
 1   state    object
dtypes: object(2)
memory usage: 34.1+ MB


In [ ]:
gmap_id_TX_uniq = gmap_id_TX.drop_duplicates(subset='gmap_id')  # Eliminar filas duplicadas basadas en 'gmap_id'
gmap_id_TX_uniq.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67740 entries, 0 to 2235403
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gmap_id  67740 non-null  object
 1   state    67740 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


In [ ]:
statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'New York';", fetch_size = 1000)
resp = session.execute(statement, timeout=None)


dictionary = {'gmap_id':[], 'state': []}


for row in resp:
  dictionary['gmap_id'].append(row['gmap_id'])
  dictionary['state'].append(row['state'])


gmap_id_NY = pd.DataFrame(dictionary)

gmap_id_NY.head()


,gmap_id,state
0,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York
1,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York
2,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York
3,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York
4,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York


In [ ]:
gmap_id_NY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2640527 entries, 0 to 2640526
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   gmap_id  object
 1   state    object
dtypes: object(2)
memory usage: 40.3+ MB


In [ ]:
gmap_id_NY.nunique()

gmap_id    70444
state          1
dtype: int64

In [ ]:
gmap_id_NY_uniq = gmap_id_NY.drop_duplicates(subset='gmap_id')  # Eliminar filas duplicadas basadas en 'gmap_id'
gmap_id_NY_uniq.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70444 entries, 0 to 2640500
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gmap_id  70444 non-null  object
 1   state    70444 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


In [ ]:
statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'Nevada';", fetch_size = 1000)
resp = session.execute(statement, timeout=None)


dictionary = {'gmap_id':[], 'state': []}


for row in resp:
  dictionary['gmap_id'].append(row['gmap_id'])
  dictionary['state'].append(row['state'])


gmap_id_NV = pd.DataFrame(dictionary)

gmap_id_NV.head()

,gmap_id,state
0,0x809838e862869fd3:0x512c619a3a2c41cc,Nevada
1,0x809838e862869fd3:0x512c619a3a2c41cc,Nevada
2,0x809838e862869fd3:0x512c619a3a2c41cc,Nevada
3,0x809838e862869fd3:0x512c619a3a2c41cc,Nevada
4,0x809838e862869fd3:0x512c619a3a2c41cc,Nevada


In [ ]:
gmap_id_NV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775999 entries, 0 to 1775998
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   gmap_id  object
 1   state    object
dtypes: object(2)
memory usage: 27.1+ MB


In [ ]:
gmap_id_NV_uniq = gmap_id_NV.drop_duplicates(subset='gmap_id')  # Eliminar filas duplicadas basadas en 'gmap_id'
gmap_id_NV_uniq.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21490 entries, 0 to 1775964
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gmap_id  21490 non-null  object
 1   state    21490 non-null  object
dtypes: object(2)
memory usage: 503.7+ KB


In [ ]:
statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'California';", fetch_size = 1000)
resp = session.execute(statement, timeout=None)


dictionary = {'gmap_id':[], 'state': []}


for row in resp:
  dictionary['gmap_id'].append(row['gmap_id'])
  dictionary['state'].append(row['state'])


gmap_id_CA = pd.DataFrame(dictionary)

gmap_id_CA.head()

,gmap_id,state
0,0x80c335e5afce634b:0xd412bd31f33038cd,California
1,0x80c335e5afce634b:0xd412bd31f33038cd,California
2,0x80c335e5afce634b:0xd412bd31f33038cd,California
3,0x80c335e5afce634b:0xd412bd31f33038cd,California
4,0x80c335e5afce634b:0xd412bd31f33038cd,California


In [ ]:
gmap_id_CA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2624746 entries, 0 to 2624745
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   gmap_id  object
 1   state    object
dtypes: object(2)
memory usage: 40.1+ MB


In [ ]:
gmap_id_CA_uniq = gmap_id_CA.drop_duplicates(subset='gmap_id')  # Eliminar filas duplicadas basadas en 'gmap_id'
gmap_id_CA_uniq.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73405 entries, 0 to 2624714
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gmap_id  73405 non-null  object
 1   state    73405 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB


In [ ]:
statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'Florida';", fetch_size = 500)
resp = session.execute(statement, timeout=None)


dictionary = {'gmap_id':[], 'state': []}


for row in resp:
  dictionary['gmap_id'].append(row['gmap_id'])
  dictionary['state'].append(row['state'])


gmap_id_FL = pd.DataFrame(dictionary)

gmap_id_FL.head()

,gmap_id,state
0,0x88dd38d385b5eb65:0xb4c00e4025ae82f3,Florida
1,0x88dd38d385b5eb65:0xb4c00e4025ae82f3,Florida
2,0x88dd38d385b5eb65:0xb4c00e4025ae82f3,Florida
3,0x88dd38d385b5eb65:0xb4c00e4025ae82f3,Florida
4,0x88dd38d385b5eb65:0xb4c00e4025ae82f3,Florida


In [ ]:
gmap_id_FL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2730490 entries, 0 to 2730489
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   gmap_id  object
 1   state    object
dtypes: object(2)
memory usage: 41.7+ MB


In [ ]:
gmap_id_FL['gmap_id'].value_counts()

0x88c2c48d32bf5d49:0x2ba1ab03c399c073    6782
0x88c2e332a3819f0d:0x285a45e522aac26f    6031
0x88e77eeacfeec8e9:0x7a463828386bb8e9    5583
0x88d9b5e001f8b66b:0xc26b69935f8af491    5352
0x88dd7f8136a04afb:0x85a2c311efe17283    4824
                                         ... 
0x88d92ec48e9cddf7:0x61588ebbd64ba959       5
0x88d904ba5ccce4f1:0x5f0cf3fe2b5e7b8c       5
0x88d91c56699d7d81:0x67b8c68ec860f38c       5
0x88c325bccdfeaa2d:0xea6a82579a811008       4
0x88d9aaf42462d131:0x7169e852b4d63902       4
Name: gmap_id, Length: 59200, dtype: int64

In [ ]:
gmap_id_FL_uniq = gmap_id_FL.drop_duplicates(subset='gmap_id')  # Eliminar filas duplicadas basadas en 'gmap_id'

In [ ]:
gmap_id_FL_uniq.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59200 entries, 0 to 2730470
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gmap_id  59200 non-null  object
 1   state    59200 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [ ]:
df_gmap_id_estates = pd.concat([gmap_id_CA_uniq, gmap_id_FL_uniq,gmap_id_NV_uniq, gmap_id_NY_uniq, gmap_id_TX_uniq ], ignore_index=True)  # Concatenar DataFrames

In [ ]:
df_gmap_id_estates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292279 entries, 0 to 292278
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   gmap_id  292279 non-null  object
 1   state    292279 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


In [ ]:
df_gmap_id_estates.nunique()

gmap_id    292279
state           5
dtype: int64

In [ ]:
df_stores_states = result = pd.merge(df_stores, df_gmap_id_estates,how = 'left', on='gmap_id')

In [ ]:
df_stores_states.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157444 entries, 0 to 157443
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   gmap_id    157444 non-null  object 
 1   name       157444 non-null  object 
 2   address    156787 non-null  object 
 3   latitude   157444 non-null  float64
 4   longitude  157444 non-null  float64
 5   misc       157444 non-null  object 
 6   state      23041 non-null   object 
dtypes: float64(2), object(5)
memory usage: 9.6+ MB


In [ ]:
df_stores_states.head()

,gmap_id,name,address,latitude,longitude,misc,state
0,0x87f38b6db3bf5557:0xb793ff26fbedb7fe,Shell,"Shell, 1510 Giant Dr, Blue Earth, MN 56013",43.657063,-94.097725,{'Accessibility': ['Wheelchair accessible entr...,NaN
1,0x80c8bf86a8b9a3f1:0xacfe57cd1919e249,Ember's Grill + Spirits,"Ember's Grill + Spirits, 740 S Rampart Blvd, L...",36.163380,-115.289413,"{'Service options': ['Delivery'], 'Highlights'...",NaN
2,0x808506614c768671:0xde51b62058db3bce,La Toque,"La Toque, 1314 McKinstry St, Napa, CA 94559",38.304173,-122.283989,"{'Service options': ['Outdoor seating', 'Dine-...",NaN
3,0x8842ecec5931b86b:0x703640198411596d,Godfather's Pizza,"Godfather's Pizza, 233 Lexington Rd, Lancaster...",37.625710,-84.580132,"{'Service options': ['Delivery'], 'Popular for...",NaN
4,0x87e030ec559cac7f:0x8f04ab05959695e6,Subway,"Subway, 330 N 1st St, Cuba, IL 61427",40.496239,-90.198875,"{'Service options': ['Curbside pickup', 'Takeo...",NaN


In [ ]:
# eliminar filas que no correspondan a los 5 estados seleccionados.
df_stores_cleaned = df_stores_states.dropna(subset=['state'])

In [ ]:
df_stores_cleaned.head()

,gmap_id,name,address,latitude,longitude,misc,state
14,0x80c2c65af79ca789:0xa585cfe5d1430d04,Pok Pok Phat Thai- Los Angeles,"Pok Pok Phat Thai- Los Angeles, 727 N Broadway...",34.061512,-118.239738,"{'Service options': ['Outdoor seating', 'Deliv...",California
16,0x8085870fc40fa211:0x5ed309bfdc595759,"Chicken'licious Noodle ""咪走鷄""","Chicken'licious Noodle ""咪走鷄"", 5801 Geary Blvd,...",37.779991,-122.481705,"{'Service options': ['Delivery'], 'Health & sa...",California
20,0x865c882f3cb0c9ab:0x47e61f2678fde77b,Shell,"Shell, 24354 Wilderness Oak, San Antonio, TX 7...",29.665701,-98.477501,{'Accessibility': ['Wheelchair accessible entr...,Texas
21,0x89e9cb13665ad3c7:0x1ad39b2f95f4f464,CJ's,"CJ's, 479 Bay Ave, Ocean Beach, NY 11770",40.648949,-73.156754,"{'Service options': ['Takeout', 'Dine-in', 'De...",New York
24,0x89c25fa8c79d0fd9:0xe8d88e50dc3328e2,Delicias Colombianas,"Delicias Colombianas, 37-03 82nd St, Queens, N...",40.749516,-73.884125,"{'Service options': ['Takeout', 'Dine-in', 'De...",New York


In [ ]:
df_stores_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23041 entries, 14 to 157442
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   gmap_id    23041 non-null  object 
 1   name       23041 non-null  object 
 2   address    23018 non-null  object 
 3   latitude   23041 non-null  float64
 4   longitude  23041 non-null  float64
 5   misc       23041 non-null  object 
 6   state      23041 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.4+ MB


In [ ]:
df_stores_cleaned.to_csv('stores_cleaned',index= False)

In [ ]:
# Función para determinar si la cadena contiene 'Delivery'
def has_delivery(row):
    if isinstance(row['misc'], str) and 'Delivery' in row['misc']:
        return 'yes'
    else:
        return 'no'

# Aplicar la función a cada fila y crear la nueva columna 'delivery'
df_stores_cleaned['delivery'] = df_stores_cleaned.apply(has_delivery, axis=1)

# Mostrar el DataFrame resultante
print(df_stores_cleaned)


                                      gmap_id                            name  \
14      0x80c2c65af79ca789:0xa585cfe5d1430d04  Pok Pok Phat Thai- Los Angeles   
16      0x8085870fc40fa211:0x5ed309bfdc595759    Chicken'licious Noodle "咪走鷄"   
20      0x865c882f3cb0c9ab:0x47e61f2678fde77b                           Shell   
21      0x89e9cb13665ad3c7:0x1ad39b2f95f4f464                            CJ's   
24      0x89c25fa8c79d0fd9:0xe8d88e50dc3328e2            Delicias Colombianas   
...                                       ...                             ...   
157409  0x866121268e434f15:0x7ba0f3a00826a54a                  Cakes of Glory   
157416  0x86456c632bc64691:0x1498937c5b72a435      Kolache Kitchen - Downtown   
157435  0x80ea695a4cc7e337:0x59115571d2177d64         Home. Boutique & Bakery   
157441  0x88e7790ad915b3bf:0xe29ffb688b004513              Echantillon Bakery   
157442  0x89c25a263dab771f:0xe92ae29003b3656d                      Q Q Bakery   

                           

<ipython-input-100-b660d3385e36>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stores_cleaned['delivery'] = df_stores_cleaned.apply(has_delivery, axis=1)


In [ ]:
df_stores_cleaned.head()

,gmap_id,name,address,latitude,longitude,misc,state,delivery
14,0x80c2c65af79ca789:0xa585cfe5d1430d04,Pok Pok Phat Thai- Los Angeles,"Pok Pok Phat Thai- Los Angeles, 727 N Broadway...",34.061512,-118.239738,"{'Service options': ['Outdoor seating', 'Deliv...",California,yes
16,0x8085870fc40fa211:0x5ed309bfdc595759,"Chicken'licious Noodle ""咪走鷄""","Chicken'licious Noodle ""咪走鷄"", 5801 Geary Blvd,...",37.779991,-122.481705,"{'Service options': ['Delivery'], 'Health & sa...",California,yes
20,0x865c882f3cb0c9ab:0x47e61f2678fde77b,Shell,"Shell, 24354 Wilderness Oak, San Antonio, TX 7...",29.665701,-98.477501,{'Accessibility': ['Wheelchair accessible entr...,Texas,no
21,0x89e9cb13665ad3c7:0x1ad39b2f95f4f464,CJ's,"CJ's, 479 Bay Ave, Ocean Beach, NY 11770",40.648949,-73.156754,"{'Service options': ['Takeout', 'Dine-in', 'De...",New York,yes
24,0x89c25fa8c79d0fd9:0xe8d88e50dc3328e2,Delicias Colombianas,"Delicias Colombianas, 37-03 82nd St, Queens, N...",40.749516,-73.884125,"{'Service options': ['Takeout', 'Dine-in', 'De...",New York,yes


In [ ]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, dict_factory

import pandas as pd

cluster = Cluster(['186.87.6.161'], port='9042', protocol_version = 5) #IP del servidor y el puerto estandar de cassandra 9042
session = cluster.connect('henry')
session.row_factory = dict_factory

'''if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

from mage_ai.data_preparation.variable_manager import set_global_variable

@data_loader
'''
# Define una variable global
reviews_yelp = None

def load_data(*args, **kwargs):
    global reviews_yelp  # Indicar que estamos utilizando la variable global

    # ... (tu código para cargar los datos)

    # Establece la variable global 'reviews_yelp' con el índice final
    reviews_yelp = indice_fin

    # Acceder a las variables globales de carga
    indice_inicio = kwargs['indice_inicio']
    tamaño_particion = kwargs['tamaño_particion']

    # Calcular el índice final
    indice_fin = indice_inicio + tamaño_particion

    statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'New York';", fetch_size = 500)
    resp = session.execute(statement, timeout=None)

    dictionary = {'gmap_id':[], 'state': [], 'user_id':[], 'name': [], 'time':[], 'rating': [], 'text':[], 'resp': []}

    for i, row in enumerate(resp):
        # Verificar si el índice actual está dentro del rango de la partición
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

    reviews_newyork = pd.DataFrame(dictionary)

    indice_fin = min(indice_fin, indice_inicio + len(reviews_newyork))

    set_global_variable('reviews_yelp', 'indice_inicio', indice_fin)


# Llama a la función load_data con los argumentos adecuados
    indice_inicio = 4000
    tamaño_particion = 5000
    load_data(indice_inicio=indice_inicio, tamaño_particion=tamaño_particion)

# Ahora puedes acceder a la variable global 'reviews_yelp' en cualquier parte del código

'''@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'
'''

'@test\ndef test_output(output, *args) -> None:\n    """\n    Template code for testing the output of the block.\n    """\n    assert output is not None, \'The output is undefined\'\n'

In [ ]:
# Define una variable global
reviews_yelp = None

def load_data(*args, **kwargs):
    global reviews_yelp  # Indicar que estamos utilizando la variable global


    # Establece la variable global 'reviews_yelp' con el índice final
    indice_inicio = kwargs['indice_inicio']
    tamaño_particion = kwargs['tamaño_particion']
    indice_fin = indice_inicio + tamaño_particion


    statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'New York';", fetch_size = 500)
    resp = session.execute(statement, timeout=None)

    dictionary = {'gmap_id':[], 'state': [], 'user_id':[], 'name': [], 'time':[], 'rating': [], 'text':[], 'resp': []}

    for i, row in enumerate(resp):
        # Verificar si el índice actual está dentro del rango de la partición
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

    reviews_newyork = pd.DataFrame(dictionary)

    indice_fin = min(indice_fin, indice_inicio + len(reviews_newyork))

    set_global_variable('reviews_yelp', 'indice_inicio', indice_fin)


    # Asigna un valor específico a 'reviews_yelp'
    reviews_yelp = indice_fin

    set_global_variable('reviews_yelp', 'indice_inicio', indice_fin)

# Llama a la función load_data con los argumentos adecuados
indice_inicio = 4000
tamaño_particion = 5000
load_data(indice_inicio=indice_inicio, tamaño_particion=tamaño_particion)

# Ahora puedes acceder a la variable global 'reviews_yelp' y asignarla a 'valor'
valor = reviews_yelp

# Imprime el valor de 'valor', que es igual al valor de 'reviews_yelp'
print(valor)


NameError: ignored

In [ ]:
# Define una variable global
reviews_yelp = None

def load_data(*args, **kwargs):
    global reviews_yelp  # Indicar que estamos utilizando la variable global

    # ... (tu código para cargar los datos)

    # Establece la variable global 'reviews_yelp' con el índice final
    indice_inicio = kwargs['indice_inicio']
    tamaño_particion = kwargs['tamaño_particion']
    indice_fin = indice_inicio + tamaño_particion

    # Resto del código para cargar los datos...

    statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'New York';", fetch_size = 500)
    resp = session.execute(statement, timeout=None)

    dictionary = {'gmap_id':[], 'state': [], 'user_id':[], 'name': [], 'time':[], 'rating': [], 'text':[], 'resp': []}

    for i, row in enumerate(resp):
        # Verificar si el índice actual está dentro del rango de la partición
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

    reviews_newyork = pd.DataFrame(dictionary)

    indice_fin = min(indice_fin, indice_inicio + len(reviews_newyork))

    # Asigna un valor específico a 'reviews_yelp'
    reviews_yelp = indice_fin

# Llama a la función load_data con los argumentos adecuados
indice_inicio = 4000
tamaño_particion = 5000
load_data(indice_inicio=indice_inicio, tamaño_particion=tamaño_particion)

# Ahora puedes acceder a la variable global 'reviews_yelp' y asignarla a 'valor'
valor = reviews_yelp

# Imprime el valor de 'valor', que es igual al valor de 'reviews_yelp'
print(valor)


KeyboardInterrupt: ignored

codigo ok


In [ ]:
@data_loader
def load_data(*args, **kwargs):

    # Acceder a las variables globales de carga
    indice_inicio = kwargs['indice_inicio']
    tamaño_particion = kwargs['tamaño_particion']

    # Calcular el índice final
    indice_fin = indice_inicio + tamaño_particion


    statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'Texas';", fetch_size = 500)
    resp = session.execute(statement, (estados,), timeout=None)

    dictionary = {'gmap_id':[], 'state': [], 'user_id':[], 'name': [], 'time':[], 'rating': [], 'text':[], 'resp': []}

    for i, row in enumerate(resp):
        # Verificar si el índice actual está dentro del rango de la partición
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

    reviews_newyork = pd.DataFrame(dictionary)

    indice_fin = min(indice_fin, indice_inicio + len(reviews_newyork))

    set_global_variable('reviewsyelp_to_sql_ale', 'indice_inicio', indice_fin)

    return reviews_newyork

@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'


In [ ]:

    categories_searched = ['Restaurant',
                        'restaurant',
                        'Bar',
                        'bar',
                        'Deli',
                        'Grocery',
                        'Coffee',
                        'Bakery',
                        'Sandwich']

    dictionary = {'gmap_id':[],
                'name': [],
                'address': [],
                'latitude': [],
                'longitude': [],
                'category':[],
                'misc':[]}



    for category_searched in categories_searched:
    # Construct the query
        query = f"""SELECT gmap_id, name, address, latitude, longitude, category, misc
                    FROM stores
                    WHERE category CONTAINS '{category_searched}'
                    """
        statement = SimpleStatement(query, fetch_size=500)
        answer = session.execute(statement, timeout=None)

        for row in answer:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['name'].append(row['name'])
            dictionary['address'].append(row['address'])
            dictionary['latitude'].append(row['latitude'])
            dictionary['longitude'].append(row['longitude'])
            # Convert the SortedSet to a comma-separated string
            category_str = ', '.join(row['category'])
            dictionary['category'].append(category_str)
            dictionary['misc'].append(row['misc'])


    stores = pd.DataFrame(dictionary)

    return stores


LLAMAR A VARIOS ESTADOS JUNTOS

In [ ]:
# Define una variable global
reviews_yelp = None

def load_data(*args, **kwargs):
    global reviews_yelp  # Indicar que estamos utilizando la variable global

    # Acceder a las variables globales de carga
    indice_inicio = kwargs['indice_inicio']
    tamaño_particion = kwargs['tamaño_particion']

    # Calcular el índice final
    indice_fin = indice_inicio + tamaño_particion

    # Variable estados
    estados = ['New York', 'California', 'Texas', 'Nevada', 'Florida']

    # Construir la consulta CQL dinámicamente
    query = "SELECT * FROM reviews WHERE state IN ("

    # Agregar marcadores de posición para cada estado
    query += ', '.join(['%s' for _ in estados])

    query += ")ALLOW FILTERING;"

    # Crear una tupla de valores para los marcadores de posición
    values = tuple(estados)

    # Ejecutar la consulta
    statement = SimpleStatement(query, fetch_size=500)
    resp = session.execute(statement, values, timeout=None)

    # Resto de tu código para cargar los datos...

# Llama a la función load_data con los argumentos adecuados
indice_inicio = 4000
tamaño_particion = 5000
load_data(indice_inicio=indice_inicio, tamaño_particion=tamaño_particion)

# Ahora puedes acceder a la variable global 'reviews_yelp' y asignarla a 'valor'
valor = reviews_yelp

# Imprime el valor de 'valor', que es igual al valor de 'reviews_yelp'
print(valor)


In [ ]:
valor = reviews_yelp  # Asigna el valor de la variable global 'reviews_yelp' a una nueva variable 'valor'
print(valor)  # Imprime el valor de 'valor', que es igual al valor de 'reviews_yelp'


None


In [ ]:
#renaming gmap_id column

df_stores_cleaned = df_stores_cleaned.rename(columns={'gmap_id': 'store_id'})

df_stores_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23041 entries, 14 to 157442
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   store_id   23041 non-null  object 
 1   name       23041 non-null  object 
 2   address    23018 non-null  object 
 3   latitude   23041 non-null  float64
 4   longitude  23041 non-null  float64
 5   misc       23041 non-null  object 
 6   state      23041 non-null  object 
 7   delivery   23041 non-null  object 
dtypes: float64(2), object(6)
memory usage: 2.1+ MB


In [ ]:
df_stores_cleaned['platform'] = 'google'

In [ ]:
df_stores_cleaned.head()

,store_id,name,address,latitude,longitude,misc,state,delivery,platform
14,0x80c2c65af79ca789:0xa585cfe5d1430d04,Pok Pok Phat Thai- Los Angeles,"Pok Pok Phat Thai- Los Angeles, 727 N Broadway...",34.061512,-118.239738,"{'Service options': ['Outdoor seating', 'Deliv...",California,yes,google
16,0x8085870fc40fa211:0x5ed309bfdc595759,"Chicken'licious Noodle ""咪走鷄""","Chicken'licious Noodle ""咪走鷄"", 5801 Geary Blvd,...",37.779991,-122.481705,"{'Service options': ['Delivery'], 'Health & sa...",California,yes,google
20,0x865c882f3cb0c9ab:0x47e61f2678fde77b,Shell,"Shell, 24354 Wilderness Oak, San Antonio, TX 7...",29.665701,-98.477501,{'Accessibility': ['Wheelchair accessible entr...,Texas,no,google
21,0x89e9cb13665ad3c7:0x1ad39b2f95f4f464,CJ's,"CJ's, 479 Bay Ave, Ocean Beach, NY 11770",40.648949,-73.156754,"{'Service options': ['Takeout', 'Dine-in', 'De...",New York,yes,google
24,0x89c25fa8c79d0fd9:0xe8d88e50dc3328e2,Delicias Colombianas,"Delicias Colombianas, 37-03 82nd St, Queens, N...",40.749516,-73.884125,"{'Service options': ['Takeout', 'Dine-in', 'De...",New York,yes,google


In [ ]:
# Reordenar las columnas moviendo la última columna a la segunda posición
new_order = ['store_id', 'platform', 'name', 'address','state','latitude'	,'longitude',	'misc',	'delivery']
df_stores_cleaned = df_stores_cleaned[new_order]
df_stores_cleaned.head()

,store_id,platform,name,address,state,latitude,longitude,misc,delivery
14,0x80c2c65af79ca789:0xa585cfe5d1430d04,google,Pok Pok Phat Thai- Los Angeles,"Pok Pok Phat Thai- Los Angeles, 727 N Broadway...",California,34.061512,-118.239738,"{'Service options': ['Outdoor seating', 'Deliv...",yes
16,0x8085870fc40fa211:0x5ed309bfdc595759,google,"Chicken'licious Noodle ""咪走鷄""","Chicken'licious Noodle ""咪走鷄"", 5801 Geary Blvd,...",California,37.779991,-122.481705,"{'Service options': ['Delivery'], 'Health & sa...",yes
20,0x865c882f3cb0c9ab:0x47e61f2678fde77b,google,Shell,"Shell, 24354 Wilderness Oak, San Antonio, TX 7...",Texas,29.665701,-98.477501,{'Accessibility': ['Wheelchair accessible entr...,no
21,0x89e9cb13665ad3c7:0x1ad39b2f95f4f464,google,CJ's,"CJ's, 479 Bay Ave, Ocean Beach, NY 11770",New York,40.648949,-73.156754,"{'Service options': ['Takeout', 'Dine-in', 'De...",yes
24,0x89c25fa8c79d0fd9:0xe8d88e50dc3328e2,google,Delicias Colombianas,"Delicias Colombianas, 37-03 82nd St, Queens, N...",New York,40.749516,-73.884125,"{'Service options': ['Takeout', 'Dine-in', 'De...",yes


In [ ]:
df_stores_cleaned.to_csv('df_stores', index=True)

In [ ]:
!pip install cassandra-driver
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, dict_factory

import pandas as pd

cluster = Cluster(['186.87.6.161'], port='9042', protocol_version = 5) #IP del servidor y el puerto estandar de cassandra 9042
session = cluster.connect('henry')
session.row_factory = dict_factory

'''if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@data_loader
'''


def load_data(*args, **kwargs):

    statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'New York';", fetch_size = 500)
    resp = session.execute(statement, timeout=None)


    dictionary = {'gmap_id':[], 'state': [], 'user_id':[], 'name': [], 'time':[], 'rating': [], 'text':[], 'resp': []}


    for row in resp:
        dictionary['gmap_id'].append(row['gmap_id'])
        dictionary['state'].append(row['state'])
        dictionary['user_id'].append(row['user_id'])
        dictionary['name'].append(row['name'])
        dictionary['time'].append(row['time'])
        dictionary['rating'].append(row['rating'])
        dictionary['text'].append(row['text'])
        dictionary['resp'].append(row['resp'])


    reviews_newyork = pd.DataFrame(dictionary)

    return reviews_newyork


'''
@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'
'''


'\n@test\ndef test_output(output, *args) -> None:\n    """\n    Template code for testing the output of the block.\n    """\n    assert output is not None, \'The output is undefined\'\n'

In [ ]:
load_data()

                                       gmap_id     state       user_id  \
0        0x89d1ab10d5e58599:0xbde28cea3afc74bf  New York  1.178222e+20   
1        0x89d1ab10d5e58599:0xbde28cea3afc74bf  New York  1.152701e+20   
2        0x89d1ab10d5e58599:0xbde28cea3afc74bf  New York  1.150746e+20   
3        0x89d1ab10d5e58599:0xbde28cea3afc74bf  New York  1.142735e+20   
4        0x89d1ab10d5e58599:0xbde28cea3afc74bf  New York  1.138342e+20   
...                                        ...       ...           ...   
2640522  0x89c25b9c0ffedda5:0xce77cc8bfde6a777  New York  1.026083e+20   
2640523  0x89c25b9c0ffedda5:0xce77cc8bfde6a777  New York  1.021010e+20   
2640524  0x89c25b9c0ffedda5:0xce77cc8bfde6a777  New York  1.019849e+20   
2640525  0x89c25b9c0ffedda5:0xce77cc8bfde6a777  New York  1.019652e+20   
2640526  0x89c25b9c0ffedda5:0xce77cc8bfde6a777  New York  1.013544e+20   

                       name                    time  rating  \
0         Colleen McDaniels 2019-07-23 18:16:14.

,gmap_id,state,user_id,name,time,rating,text,resp
0,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York,1.178222e+20,Colleen McDaniels,2019-07-23 18:16:14.035,5,Great organization,None
1,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York,1.152701e+20,Lucas Calafiore,2020-06-23 21:08:59.278,5,None,None
2,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York,1.150746e+20,Thomas Testani,2019-04-22 17:56:17.739,4,None,None
3,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York,1.142735e+20,mad mike,2018-10-20 23:11:10.324,5,None,None
4,0x89d1ab10d5e58599:0xbde28cea3afc74bf,New York,1.138342e+20,Reggie Hoffman,2020-12-27 12:37:05.099,5,People were very friendly and enjoy our self.,None
...,...,...,...,...,...,...,...,...
2640522,0x89c25b9c0ffedda5:0xce77cc8bfde6a777,New York,1.026083e+20,Kevin Lowe,2018-12-20 14:25:52.083,5,very very neice guy and show good hospitality,None
2640523,0x89c25b9c0ffedda5:0xce77cc8bfde6a777,New York,1.021010e+20,Jai Berg,2018-08-15 19:05:01.201,4,None,None
2640524,0x89c25b9c0ffedda5:0xce77cc8bfde6a777,New York,1.019849e+20,Lanette Brightwell,2020-12-13 05:59:28.723,4,"Great Paint store, excellent handwriting",None
2640525,0x89c25b9c0ffedda5:0xce77cc8bfde6a777,New York,1.019652e+20,Yioula Euthimaki,2018-11-23 03:10:34.621,1,None,None


In [ ]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, dict_factory
import set_global_variable
import pandas as pd

cluster = Cluster(['186.87.6.161'], port='9042', protocol_version = 5) #IP del servidor y el puerto estandar de cassandra 9042
session = cluster.connect('henry')
session.row_factory = dict_factory

'''if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

from mage_ai.data_preparation.variable_manager import set_global_variable

@data_loader'''
def load_data(*args, **kwargs):

    # Acceder a las variables globales de carga
    indice_inicio = kwargs['indice_inicio']
    tamaño_particion = kwargs['tamaño_particion']

    # Calcular el índice final
    indice_fin = indice_inicio + tamaño_particion

    statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'California';", fetch_size = 500)
    resp = session.execute(statement, timeout=None)

    dictionary = {'gmap_id':[], 'state': [], 'user_id':[], 'name': [], 'time':[], 'rating': [], 'text':[], 'resp': []}

    for i, row in enumerate(resp):
        # Verificar si el índice actual está dentro del rango de la partición
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

    reviews_california = pd.DataFrame(dictionary)

    indice_fin = min(indice_fin, indice_inicio + len(reviews_california))

    set_global_variable('prueba_cassandra_to_sql', 'indice_inicio', indice_fin)

    return reviews_california

'''@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'
'''

ModuleNotFoundError: ignored

In [ ]:
from mage_ai.settings.repo import get_repo_path
from mage_ai.io.config import ConfigFileLoader
from mage_ai.io.mysql import MySQL
from pandas import DataFrame
from os import path

if 'data_exporter' not in globals():
    from mage_ai.data_preparation.decorators import data_exporter


@data_exporter
def export_data_to_mysql(df: DataFrame, **kwargs) -> None:
    """
    Template for exporting data to a MySQL database.
    Specify your configuration settings in 'io_config.yaml'.

    Docs: https://docs.mage.ai/design/data-loading#mysql
    """
    table_name = 'reviews_newyork'  # Specify the name of the table to export data to
    config_path = path.join(get_repo_path(), 'io_config.yaml')
    config_profile = 'default'

    with MySQL.with_config(ConfigFileLoader(config_path, config_profile)) as loader:
        loader.export(
            df,
            None,
            table_name,
            index=False,  # Specifies whether to include index in exported table
            if_exists='replace',  # Specify resolution policy if table name already exists
        )

### carga de todos los estados


In [ ]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, dict_factory

import pandas as pd

cluster = Cluster(['186.87.6.161'], port='9042', protocol_version = 5) #IP del servidor y el puerto estandar de cassandra 9042
session = cluster.connect('henry')
session.row_factory = dict_factory

if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

from mage_ai.data_preparation.variable_manager import set_global_variable

@data_loader
def load_data(*args, **kwargs):

    # Acceder a las variables globales de carga
    indice_inicio = kwargs['indice_inicio']
    particion = kwargs['particion']

    # Calcular el índice final
    indice_fin = indice_inicio + particion

    statement = SimpleStatement("SELECT * FROM reviews WHERE state = 'California';", fetch_size = 500)
    resp = session.execute(statement, timeout=None)

    dictionary = {'gmap_id':[], 'state': [], 'user_id':[], 'name': [], 'time':[], 'rating': [], 'text':[], 'resp': []}

    for i, row in enumerate(resp):
        # Verificar si el índice actual está dentro del rango de la partición
        if indice_inicio <= i < indice_fin:
            dictionary['gmap_id'].append(row['gmap_id'])
            dictionary['state'].append(row['state'])
            dictionary['user_id'].append(row['user_id'])
            dictionary['name'].append(row['name'])
            dictionary['time'].append(row['time'])
            dictionary['rating'].append(row['rating'])
            dictionary['text'].append(row['text'])
            dictionary['resp'].append(row['resp'])

    reviews_california = pd.DataFrame(dictionary)

    indice_fin = min(indice_fin, indice_inicio + len(reviews_california))

    set_global_variable('reviews_google_to_sql', 'indice_inicio', indice_fin)

    return reviews_california

@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'

In [ ]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement, dict_factory

import pandas as pd

cluster = Cluster(['186.87.6.161'], port='9042', protocol_version = 5) #IP del servidor y el puerto estandar de cassandra 9042
session = cluster.connect('henry')
session.row_factory = dict_factory

if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test

from mage_ai.data_preparation.variable_manager import set_global_variable

@data_loader
def load_data(*args, **kwargs):

    # Acceder a las variables globales de carga
    indice_inicio = kwargs['indice_inicio']
    tamaño_particion = kwargs['tamaño_particion']

    # Calcular el índice final
    indice_fin = indice_inicio + tamaño_particion

    Estados = ['New York','Texas', 'Nevada', 'California', 'Florida']

    dictionary = {
                'gmap_id':[],
                'state': [],
                'user_id':[],
                'name': [],
                'time':[],
                'rating': [],
                'text':[],
                'resp': []}

    for estado in Estados:
    # Construct the query
        query = f"""SELECT gmap_id, state, user_id, name, time, rating, text, resp
                    FROM reviews WHERE state = '{estado}'
                    """
        statement = SimpleStatement(query,fetch_size = 500)
        resp = session.execute(statement,timeout=None)


        for i, row in enumerate(resp):
            # Verificar si el índice actual está dentro del rango de la partición
            if indice_inicio <= i < indice_fin:
                dictionary['gmap_id'].append(row['gmap_id'])
                dictionary['state'].append(row['state'])
                dictionary['user_id'].append(row['user_id'])
                dictionary['name'].append(row['name'])
                dictionary['time'].append(row['time'])
                dictionary['rating'].append(row['rating'])
                dictionary['text'].append(row['text'])
                dictionary['resp'].append(row['resp'])

    reviews_texas = pd.DataFrame(dictionary)

    indice_fin = min(indice_fin, indice_inicio + len(reviews_texas))

    set_global_variable('reviewsyelp_to_sql_ale', 'indice_inicio', indice_fin)

    return reviews_texas

@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'